# Adaptive PDE discretizations on cartesian grids 
## Volume : Algorithmic tools
## Part : Automatic differentiation
## Chapter : Known bugs and incompatibilities

The techniques of automatic differentiation technique play an essential role in the notebooks presented in this repository. 
Our library is based on subclassing the `numpy.ndarray` class, and is written entirely in Python. This allows for a simple and powerfull implementation, which benefits from the high performance of the numpy module. It does however suffer from a few pitfalls, briefly described below, and illustrated in more detail in the body of the document.

**! Caution with the functions np.sort, np.where, np.stack, np.broadcast_to !**
* Problem : the arguments are silently cast to np.ndarray, loosing autodiff information.
* Solution : use similarly named replacements from the AutomaticDifferentiation (ad) library, which also apply to np.ndarray.

**! Caution with numpy scalars and array scalars !**
* Problem. In an expression `a+b` where the l.h.s is a numpy scalar, and the r.h.s an array scalar of autodiff type, the r.h.s is silently cast loosing autodiff information.
* Solution : use `b+a` instead, or `ad.toarray(a)+b`. 

[**Summary**](Summary.ipynb) of volume Algorithmic tools, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. *Non-universal* functions from the numpy module](#1.-*Non-universal*-functions-from-the-numpy-module)
    * [1.1 Numpy universal functions](#1.1-Numpy-universal-functions)
    * [1.2 Failure examples](#1.2-Failure-examples)
  * [2. The problem with numpy scalars on the left of array scalars](#2.-The-problem-with-numpy-scalars-on-the-left-of-array-scalars)
    * [2.1 Basic case](#2.1-Basic-case)
    * [2.2 Unexpected occurences](#2.2-Unexpected-occurences)
    * [2.3 Solution with a trailing singleton dimension](#2.3-Solution-with-a-trailing-singleton-dimension)
    * [2.4 Matrix multiplication and inversion](#2.4-Matrix-multiplication-and-inversion)
  * [3. In place modifications and aliasing](#3.-In-place-modifications-and-aliasing)
    * [3.1 Aliasing of the AD information](#3.1-Aliasing-of-the-AD-information)
    * [3.1 Non writeable AD information](#3.1-Non-writeable-AD-information)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow importing agd from parent directory
#from Miscellaneous import TocTools; TocTools.displayTOC('ADBugs','Algo')

In [2]:
import numpy as np
import scipy.sparse.linalg

In [3]:
import agd.AutomaticDifferentiation as ad

In [4]:
def reload_packages():
    import importlib
    ad = importlib.reload(sys.modules['agd.AutomaticDifferentiation'])
    ad.reload_submodules()

## 1. *Non-universal* functions from the numpy module

Automatic differentiation is based on replacing, as transparently as possible, the base variable type - in our case `np.ndarray` - with a type that incorporate additional information. 
We have chosen new type to be a *subclass* of the `np.ndarray` class, so as to preserve as much functionality of the numpy library. Of course, many functions still need to be rewritten in order to take into account the additional information contained in the subclass.


### 1.1 Numpy universal functions

The numpy library has introduced a mechanism, known as universal functions, which to put it simply replaces the function call `np.sqrt(a)` with `a.sqrt()`, and likewise for many other methods. See the manual [link](https://docs.scipy.org/doc/numpy/reference/ufuncs.html) (not all are implemented here).

If the subclass implements an adequate `sqrt` method, taking into account the AD information in our case, then it will be transparently called.

**Special math functions**

In [5]:
x=ad.Dense.identity(constant=np.array(1.))

In [6]:
print(np.sqrt(x),"equals", x.sqrt())
print(np.sin(x),"equals", x.sin())
print(np.exp(x),"equals", x.exp())

denseAD(1.0,[0.5]) equals denseAD(1.0,[0.5])
denseAD(0.8414709848078965,[0.54030231]) equals denseAD(0.8414709848078965,[0.54030231])
denseAD(2.718281828459045,[2.71828183]) equals denseAD(2.718281828459045,[2.71828183])


**Reductions.** The universal function concept is not limited to special mathematical functions. It also extends to reductions via the minimum, maximum, or sum for instance.

In [7]:
reload_packages()

In [8]:
x=ad.Dense.identity(constant=np.array([1.,2.,3.,4.]))**2

In [9]:
print(np.sum(x),"equals",x.sum())
print(np.min(x),"equals",x.min())
print(np.max(x),"equals",x.max())

denseAD(30.0,[2. 4. 6. 8.]) equals denseAD(30.0,[2. 4. 6. 8.])
denseAD(1.0,[2. 0. 0. 0.]) equals denseAD(1.0,[2. 0. 0. 0.])
denseAD(16.0,[0. 0. 0. 8.]) equals denseAD(16.0,[0. 0. 0. 8.])


**Arithmetic operations.** Arithmetic operations on a subclass of `np.ndarray` can be handled in two ways: either as a 'magic' *Python* operator, or as a function of the *numpy module*.

In [10]:
x=ad.Dense.identity(constant=np.array([1.,2.]))**2

In [11]:
print("Call to x.__add__ (python magic function) :",x+1)
x+=1.; print("Call to x.add (numpy universal function) :",x)

Call to x.__add__ (python magic function) : denseAD([2. 5.],
[[2. 0.]
 [0. 4.]])
Call to x.add (numpy universal function) : denseAD([2. 5.],
[[2. 0.]
 [0. 4.]])


### 1.2 Failure examples

Subclassing from `np.ndarray` unfortunately has a few pitfalls, as some functions often 
* *Malfunction*. For instance `np.sort` does nothing on ad types.
* *Delete* the additional information, silently, by casting the arguments to the base `np.ndarray` class. This is arguably the worst that can happen.

There is no way (known to the author) to circumvent this defective behavior.
It happens because not all numpy function adhere to the universal function concept.
In other languages, such as C++, this issue could be avoided using function overloading.

**! Malfunction : `np.sort` function does nothing !** Use the `ad.sort` function instead.

In [12]:
x=ad.Dense.identity(constant=np.array([4.,1.,3.,2.]))**2

In [13]:
print("ad.sort :", ad.sort(x))
print("np.sort (Malfunction : does nothing) :", np.sort(x))

ad.sort : denseAD([ 1.  4.  9. 16.],
[[0. 2. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 6. 0.]
 [8. 0. 0. 0.]])
np.sort (Malfunction : does nothing) : denseAD([16.  1.  9.  4.],
[[8. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 6. 0.]
 [0. 0. 0. 4.]])


**! Deletion of AD information : np.where, np.stack, np.broadcast_to silently cast to base class !**
Some functions numpy functions will cast their arguments to the base class `np.ndarray`.
Their variant from the `ad` library needs to be called.

In [14]:
print("ad.where :",ad.where(x<2.,0.,x))
print("np.where (Deletion : silent cast to base) :", np.where(x<2.,0.,x))

ad.where : denseAD([16.  0.  9.  4.],
[[8. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 6. 0.]
 [0. 0. 0. 4.]])
np.where (Deletion : silent cast to base) : [16.  0.  9.  4.]


In [15]:
print("ad.broadcast_to :",ad.broadcast_to(x,(2,4)))
print("np.broadcast_to (Deletion : silent cast to base) :",np.broadcast_to(x,(2,4)))

ad.broadcast_to : denseAD([[16.  1.  9.  4.]
 [16.  1.  9.  4.]],
[[[8. 0. 0. 0.]
  [0. 2. 0. 0.]
  [0. 0. 6. 0.]
  [0. 0. 0. 4.]]

 [[8. 0. 0. 0.]
  [0. 2. 0. 0.]
  [0. 0. 6. 0.]
  [0. 0. 0. 4.]]])
np.broadcast_to (Deletion : silent cast to base) : [[16.  1.  9.  4.]
 [16.  1.  9.  4.]]


In [16]:
print("ad.stack:",ad.stack((x,2*x)))
print("np.stack (Deletion : silent cast to base) :",np.stack((x,2*x)))

ad.stack: denseAD([[16.  1.  9.  4.]
 [32.  2. 18.  8.]],
[[[ 8.  0.  0.  0.]
  [ 0.  2.  0.  0.]
  [ 0.  0.  6.  0.]
  [ 0.  0.  0.  4.]]

 [[16.  0.  0.  0.]
  [ 0.  4.  0.  0.]
  [ 0.  0. 12.  0.]
  [ 0.  0.  0.  8.]]])
np.stack (Deletion : silent cast to base) : [[16.  1.  9.  4.]
 [32.  2. 18.  8.]]


## 2. The problem with numpy scalars on the left of array scalars

**! Caution with numpy scalars and autodiff array scalars !**

The type `numpy.float64` often causes trouble due to bad operator priority. Specifically, when it is multiplied (when multiplied with an array of shape `()` and containing automatic differentiation information). We circumvent this issue using the function ad.to_array which casts any value of to a numpy array, in this case to an array containing a single element and of shape $()$ (the empty tuple).

**Context** In order to discuss this issue, which occurs in very specific circumstances, we need to introduce a few concepts.
* A numpy scalar is a variable of type `numpy.float64`, or possibly some other integer of floating point type defined in the numpy module. Standard python scalars, such as `float` and `int`, are not affected by the issue below.
* An array scalar is an array whose shape is the empty tuple `()`. Such arrays contain a single element, and for most purposes behave like a scalar variable.
* Operator resolution is the process by which Python selects the appropriate function to compute `a+b` where `a` and `b` are two variables. In practice: 
 * Python first calls `a.__add__(b)`. 
 * The result is returned, except if it is the special value `NotImplemented`.
 * In that case Python calls `b.__radd__(a)` (note the 'r' which stands for 'right' side operator).

**The problem.**
If `a` is of type `numpy.float64`, and `b` is a subclass of `np.ndarray`, then `a.__add__(b)` usually returns `NotImplemented`, and is superseded by the adequate `b.__radd__(a)`. The exception to the rule is when is an array scale. In that case, `b` is cast to the based class `np.ndarray` loosing all AD information, and its (single) value is added to `a`.

**Solution.**
The idea is to avoid is previous situation, either exchanging the lhs and rhs, or by an appropriate casting.




### 2.1 Basic case

let us illustrate the problem in its most basic form, with simple scalars.

In [17]:
a = np.float64(1.)
b = ad.Dense.identity(constant=np.array(1.)) 
print("a =",a,", b =",b)
print("Error (cast to numpy scalar). a+b =",a+b)

a = 1.0 , b = denseAD(1.0,[1.])
Error (cast to numpy scalar). a+b = 2.0


In [18]:
print("b is an array scalar : b.shape =",b.shape)

b is an array scalar : b.shape = ()


Several solutions can be used.

In [19]:
print(b+a) # Simplest
print(ad.toarray(a)+b)
print(ad.toarray(a)+ad.toarray(b)) #Safest

denseAD(2.0,[1.])
denseAD(2.0,[1.])
denseAD(2.0,[1.])


The same issue arises with the other arithmetic operators

In [20]:
print("Error (cast to numpy scalar). a-b =",a-b)
print("Error (cast to numpy scalar). a*b =",a*b)
print("Error (cast to numpy scalar). a/b =",a/b)

Error (cast to numpy scalar). a-b = 0.0
Error (cast to numpy scalar). a*b = 1.0
Error (cast to numpy scalar). a/b = 1.0


The same solutions apply. Which one is the simplest can be discussed for non-symmetric operators.

In [21]:
print(-(b-a)) 
print(ad.toarray(a)-b) # Simplest
print(ad.toarray(a)-ad.toarray(b)) #Safest
print()

print(b*a) # Simplest
print(ad.toarray(a)*b)
print(ad.toarray(a)*ad.toarray(b)) #Safest
print()

print(1./(b/a))
print(ad.toarray(a)/b)  # Simplest
print(ad.toarray(a)/ad.toarray(b)) #Safest

denseAD(-0.0,[-1.])
denseAD(-0.0,[-1.])
denseAD(-0.0,[-1.])

denseAD(1.0,[1.])
denseAD(1.0,[1.])
denseAD(1.0,[1.])

denseAD(1.0,[-1.])
denseAD(1.0,[-1.])
denseAD(1.0,[-1.])


All these problems disappear if 'b' is anything else than an array scalar. In other words if `b.shape!=()`.

In [22]:
a = np.float64(1.)
b = ad.Dense.identity(constant=np.array([1.])) 
print("b is not an array scalar : b.shape =", b.shape)

b is not an array scalar : b.shape = (1,)


In [23]:
print("a+b =",a+b)
print("a-b =",a-b)
print("a*b =",a*b)
print("a/b =",a/b)

a+b = denseAD([2.],[[1.]])
a-b = denseAD([-0.],[[-1.]])
a*b = denseAD([1.],[[1.]])
a/b = denseAD([1.],[[-1.]])


### 2.2 Unexpected occurences

The problem depicted above may infortunately occur in a slightly hidden form, where may not thinking about numpy scalars and array scalars.

In [24]:
a = np.array([1.,2.,3.])
b = ad.Dense.identity(constant=np.array([4.,5.,6.]))

In [25]:
print("a[0] is a numpy scalar.", type(a[0]))
print("a.sum() is a numpy scalar.", type(a.sum()))
print()

print("b[0] is an array scalar.", b[0].shape)
print("b.sum() is an array scalar.",b.sum().shape)

a[0] is a numpy scalar. <class 'numpy.float64'>
a.sum() is a numpy scalar. <class 'numpy.float64'>

b[0] is an array scalar. ()
b.sum() is an array scalar. ()


In [26]:
print("Error (cast to numpy scalar).",a[0]+b[0])
print("Error (cast to numpy scalar).",a.sum()+b.sum())

Error (cast to numpy scalar). 5.0
Error (cast to numpy scalar). 21.0


In the following example, an incorrect value is assigned.

In [27]:
B=b.copy(); B[0]=a[0]+B[0]; print("Incorrect (AD information lost).", B[0])

Incorrect (AD information lost). denseAD(5.0,[0. 0. 0.])


The previous solutions apply.

In [28]:
print(b[0]+a[0]) # Simplest
print(ad.toarray(a[0])+b[0])
print(ad.toarray(a[0])+ad.toarray(b[0])) # Safest
print() 

print(ad.toarray(a.sum())+b.sum())
print()

B=b.copy(); B[0]=ad.toarray(a[0])+B[0]; print(B[0])

denseAD(5.0,[1. 0. 0.])
denseAD(5.0,[1. 0. 0.])
denseAD(5.0,[1. 0. 0.])

denseAD(21.0,[1. 1. 1.])

denseAD(5.0,[1. 0. 0.])


However, alternative approaches can be considered too. For instance in the assignement case.

In [29]:
B=b.copy(); B[0]+=a[0]; print(B[0]) # Using in place assigment
B=b.copy(); B[[0]]=a[[0]]+B[[0]]; print(B[0]) # Using non-scalar arrays

denseAD(5.0,[1. 0. 0.])
denseAD(5.0,[1. 0. 0.])


A radical solution is to convert `a` to a variable `A` of AD type. 

In [30]:
A = ad.toarray(a,type(b)) # A is the conversion of a to the same type as b
print("A[0] is an array scalar (not a numpy scalar).", type(A[0]), A[0].shape)

A[0] is an array scalar (not a numpy scalar). <class 'NumericalSchemes.AutomaticDifferentiation.Dense.denseAD'> ()


The cost is minor, the AD information of $A$ only consists of an empty array.

In [31]:
print(A.size_ad)
print(A.coef.size)

0
0


In [32]:
print(A[0]+b[0])
print(A.sum()+b.sum())
B=b.copy(); B[0]=A[0]+B[0]; print(B[0])

denseAD(5.0,[1. 0. 0.])
denseAD(21.0,[1. 1. 1.])
denseAD(5.0,[1. 0. 0.])


Finally, let us recall that the above problems disappear in the case of non-scalar arrays.

In [33]:
a[0:2]+b[0:2]

denseAD(array([5., 7.]),
array([[1., 0., 0.],
       [0., 1., 0.]]))

### 2.3 Solution with a trailing singleton dimension

Yet another solution is to fully eliminate array scalars of AD type, by introducing a (e.g. trailing) singleton dimension. This solution requires a bit of code refactoring, but should be transparent in most places.

In [34]:
a = np.float64(1)
b = ad.Dense.identity(constant=np.array([4.,5.,6.]))
b = np.expand_dims(b,axis=-1) # Add a trailing singleton dimension

In [35]:
a+b[0] # Problem solved

denseAD(array([5.]),array([[1., 0., 0.]]))

### 2.4 Matrix multiplication and inversion

A similar issue arises with matrix multiplication and inversion : the AD information is lost. An appropriate syntax, presented below, allows to preserve it.


In [174]:
v = ad.Dense.denseAD( np.random.standard_normal((4,)),np.random.standard_normal((4,4)))
m0 = np.random.standard_normal((4,4))
m1 = scipy.sparse.coo_matrix( ([1.,2.,3.,4.,5.],([0,2,1,2,3],[0,1,2,2,3]))).tocsr()

In [169]:
print("np.dot looses AD:",np.dot(m0,v))
print("scipy '*' looses AD:",m1*v.value)

np.dot looses AD: [ 1.70530847 -0.11456795 -0.73066176  1.48267191]
scipy '*' looses AD: [1.67581874 2.97884129 3.97178838 0.20522961]


In [180]:
print("np.dot with AD:\n",ad.apply_linear_mapping(m0,v))
print("scipy '*' with AD:\n",ad.apply_linear_mapping(m1,v))

np.dot with AD:
 denseAD([-1.89150544  0.3796543   1.30376865 -0.98564952],
[[-1.6848455  -2.83764912  1.77247252 -2.77079864]
 [-2.20391131 -0.56107039  1.68847524 -2.58272018]
 [ 1.01843512 -1.36934636  0.89630998 -1.31008306]
 [-2.03349406 -1.29100599  1.54521456 -2.07800914]])
scipy '*' with AD:
 denseAD([ 0.8778869   1.1385308   2.85995405 -7.16854102],
[[-0.34111709 -0.14956627  0.64471031 -1.03636975]
 [ 3.67738763 -0.04489892 -4.54696905 -1.30243209]
 [ 5.89038497 -2.86303733 -4.29064566 -4.47909757]
 [-5.90443925 -4.00817189 -1.08642606 -7.62422685]])


In [179]:
print("scipy solve with AD :\n",ad.apply_linear_inverse(scipy.sparse.linalg.spsolve,m1,v))

scipy solve with AD :
 denseAD([ 0.8778869  -0.25754919  0.22365216 -0.28674164],
[[-0.34111709 -0.14956627  0.64471031 -1.03636975]
 [ 0.28383078  0.92690755 -1.34848809  0.6971018 ]
 [ 0.16453358 -0.46719535  0.29532996 -0.45708691]
 [-0.23617757 -0.16032688 -0.04345704 -0.30496907]])


## 3. In place modifications and aliasing

The AD information often consists of very large arrays. In order to save time and memory, this information is not systematically copied and/or stored fully. It can take the form of a broadcasted array, or of an alias to another array. In that case a copy is necessary to enable modifications.

### 3.1 Aliasing of the AD information

When an operation leaves the AD information untouched, an alias is used. This can lead to bugs if in place modifications are used afterward.

In [36]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=x+1 # Only affects the value, not the AD information

In [37]:
print("Values are distinct :", x.value is y.value)
print("AD information is shared :", y.coef is x.coef)

Values are distinct : False
AD information is shared : True


A modification of the aliased variable will impact the original one.

In [38]:
print(x[0])
y[0]*=2
print("Caution ! Shared AD information is affected :", x[0])

denseAD(1.0,[1. 0.])
Caution ! Shared AD information is affected : denseAD(1.0,[2. 0.])


Avoid this effect by making a copy.

In [39]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=(x+1).copy()
print("AD information is distinct :", y.coef is x.coef)

AD information is distinct : False


Note that a similar effect arises with the `-` binary operator, but not with `*`or `/`. That is because the latter modify the AD information, which therefore must be copied anyway.

In [40]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
print("AD information is shared :", (x-1).coef is x.coef)
print("AD information is distinct :", (x*2).coef is x.coef)
print("AD information is distinct :", (x/2).coef is x.coef)

AD information is shared : True
AD information is distinct : False
AD information is distinct : False


### 3.1 Non writeable AD information

When creating an dense AD variable, the coefficients may be non writeable (e.g. broadcasted) arrays.

In [41]:
x=ad.Dense.identity(constant=np.array([[1.,2.],[3.,4.]]),shape_bound=(2,))

In [42]:
x.coef.flags.writeable

False

In [43]:
# x+=1 # Fails because non-writeable

Make a copy to solve the issue.

In [44]:
y=x.copy()

In [45]:
y.coef.flags.writeable

True

In [46]:
y+=1